## 7-1 気象データによる電力消費量予測

### ライブラリーのインポート

In [ ]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
%matplotlib inline

- sklearn.model_selection.KFold
  - k-分割交差検証のライブラリ

### データの読み込み

In [ ]:
data = pd.read_csv('data/weather-elec_data.csv',
                   index_col = '日時', parse_dates=True)
data.sample(5)

- data = pd.read_csv('ファイル名')
  - csvファイルを読み込んで、データフレームへ格納する。
  - index_col = '日時': 「日時」列をインデックスに設定する。
  - parse_dates=True: インデックスをdatetime型に変換する。
- data.sample(5)
  - ランダムに5つのデータを表示する。

### 日時の分割

In [ ]:
# 日時を年・月・日・時に分割
data['月'] = data.index.month
data['日'] = data.index.day
data['時'] = data.index.hour
ml_data = data.reset_index(drop=True)
ml_data.sample(5)

- data['月'] = data.index.month
  - インデックスから「月」のデータを切り出して、データ['月']へ代入する。
  - data['日'] = 〜 、data['時'] = 〜 も同様の処理
- ml_data = data.reset_index(drop=True)
  - インデックスを削除して、機械学習用データ（ml_data）とする。

### 学習の準備

In [ ]:
# 特徴量（X）と予測対象（y）の設定
X = ml_data[['気温', '日照時間']].values
y = ml_data[['実績']].values
# k分割交差検証の準備
kf = KFold(n_splits=5, shuffle=True, random_state=0)

- X = 〜 : 気温と日照時間の値を特徴量として取り出す。
- y = 〜 : 実績（電力量）を正解データとして取り出す。
- kf = KFold(n_splits=5, shuffle=True, random_state=0)
  - 5分割交差検証用のデータジェネレータ（kf）を設定する。
  - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

### 学習の実行

In [ ]:
cnt = score = 0
for train, test in kf.split(X):
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]

    # 学習モデルの生成
    model = svm.SVR(gamma = 'auto')
    y_train = y_train.flatten()
    y_test = y_test.flatten()

    # 学習の実行
    cnt += 1
    print('Executing', cnt)
    model.fit(X_train, y_train)
    train_score = model.score(X_train, y_train)
    test_score  = model.score(X_test, y_test)
    score += test_score
    print('+ SVR: Training Score = %f, Test Score = %f' 
          % (train_score, test_score))

- for train, test in kf.split(X):
  - kf.split(X)：訓練データテストデータにに分割するためのインデックスを生成する。
  - for文で分割数回の処理を繰り返す。
- model = svm.SVR(gamma = 'auto')
  - svm.SVR（回帰モデル)のインスタンスを生成する。
- model.fit(X_train, y_train)
  - 訓練データを使って機械学習を実行する。
- model.score( ･･･ , ･･･ )
  - 評価指標（決定係数）の計算を行う（訓練データ／テストデータ）

### 学習済みモデルの評価

In [ ]:
avr_score = score / cnt
print ('Average Test Score = %f' % avr_score)

- avr_score = score / cnt
  - 評価指標（決定係数）の平均値を計算する。

### 実測値と予測値の相関グラフ

In [ ]:
plt.figure(figsize=(10, 6))
pred = model.predict(X)
plt.scatter(y, pred, s=0.5, color="black")
plt.xlabel('electricity consumption(measured *10000 kW)')
plt.ylabel('electricity consumption(predicted *10000 kW)')
plt.show()

- 横軸：電力消費量の実測値、縦軸: 電力消費量の予測値
- 実測と予測が完全に一致すると直線になる。